## Dependencies

In [1]:
!pip uninstall -y tensorflow numpy && pip install tensorflow-gpu==1.14.0 tf_sentencepiece

Uninstalling tensorflow-1.14.0:
  Successfully uninstalled tensorflow-1.14.0
Uninstalling numpy-1.16.5:
  Successfully uninstalled numpy-1.16.5
     |████████████████████████████████| 377.0MB 49kB/s 
     |████████████████████████████████| 2.7MB 27.2MB/s 
     |████████████████████████████████| 20.4MB 30.6MB/s 
ERROR: stable-baselines 2.2.1 requires tensorflow>=1.5.0, which is not installed.
ERROR: magenta 0.3.19 requires tensorflow>=1.12.0, which is not installed.
ERROR: fancyimpute 0.4.3 requires tensorflow, which is not installed.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [2]:
!curl -o many_to_many.zip https://dl.dropboxusercontent.com/s/kzxryp6snnx5dw6/many_to_many.zip?dl=0

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  114M  100  114M    0     0  39.0M      0  0:00:02  0:00:02 --:--:-- 39.0M


In [0]:
!unzip -q many_to_many.zip

In [4]:
!curl -o mlt.zip https://codeload.github.com/suyash/mlt/zip/master

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 33661    0 33661    0     0   151k      0 --:--:-- --:--:-- --:--:--  151k


In [0]:
!unzip -q mlt.zip && mv mlt-master/mlt ./

## Many to Many Translation

In [0]:
import numpy as np
import tensorflow as tf
import tf_sentencepiece as tfs

from mlt.evaluation import predict
from mlt.layers import Attention, ConditionalNormalization, Gelu, MultiplyConstant, PaddingAndLookaheadMask, PaddingMask, PositionalEncoding

In [7]:
tf.__version__

'1.14.0'

In [0]:
sess = tf.InteractiveSession()

In [0]:
encoding_model_file = "./final/sentencepiece/en_de_fr_pt_shared/encoding.model"

In [10]:
with sess.as_default():
    model = tf.keras.experimental.load_from_saved_model("final/model114", custom_objects={
        "MultiplyConstant": MultiplyConstant,
        "PositionalEncoding": PositionalEncoding,
        "PaddingMask": PaddingMask,
        "PaddingAndLookaheadMask": PaddingAndLookaheadMask,
        "Attention": Attention,
        "ConditionalNormalization": ConditionalNormalization,
        "Gelu": Gelu,
    })

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


### English to French

In [0]:
a = tfs.encode(["This is a problem that we need to solve."], model_file=encoding_model_file, add_bos=True, add_eos=True)[0]

In [12]:
ids, probs = predict(
    model=model,
    inputs=a,
    inpf=tf.constant([1.0, 0.0, 0.0, 0.0]),
    tarf=tf.constant([0.0, 1.0, 0.0, 0.0]),
    bos_id=1,
    eos_id=2,
    beam_size=5,
    vocab_size=40960,
    alpha=1.0,
)

Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
mask = tf.cast(tf.not_equal(ids, 0), tf.int32)
seq_len = tf.reduce_sum(mask, axis=-1)
probs = tf.math.exp(probs)

In [0]:
ids_, probs_, seq_len_ = sess.run([ids, probs, seq_len])

In [15]:
[x.decode("utf-8") for x in sess.run(tfs.decode(ids_[0], seq_len_[0], model_file=encoding_model_file))], probs_[0]

(["Il s'agit d'un problème que nous devons résoudre.",
  "C'est un problème que nous devons résoudre.",
  'C’est un problème que nous devons résoudre.',
  'Ce problème est que nous devons résoudre.',
  "Il s'agit d'un problème que nous avons besoin de résoudre."],
 array([0.35213375, 0.33331218, 0.2737111 , 0.2399587 , 0.21996328],
       dtype=float32))

### English to German

In [0]:
a = tfs.encode(["This is a problem that we need to solve."], model_file=encoding_model_file, add_bos=True, add_eos=True)[0]

In [0]:
ids, probs = predict(
    model=model,
    inputs=a,
    inpf=tf.constant([1.0, 0.0, 0.0, 0.0]),
    tarf=tf.constant([0.0, 0.0, 1.0, 0.0]),
    bos_id=1,
    eos_id=2,
    beam_size=5,
    vocab_size=40960,
    alpha=1.0,
)

In [0]:
mask = tf.cast(tf.not_equal(ids, 0), tf.int32)
seq_len = tf.reduce_sum(mask, axis=-1)
probs = tf.math.exp(probs)

In [0]:
ids_, probs_, seq_len_ = sess.run([ids, probs, seq_len])

In [20]:
[x.decode("utf-8") for x in sess.run(tfs.decode(ids_[0], seq_len_[0], model_file=encoding_model_file))], probs_[0]

(['Das ist ein Problem, das wir lösen müssen.',
  'Dies ist ein Problem, das wir lösen müssen.',
  'Das ist ein Problem, der wir lösen müssen.',
  'Dies ist ein Problem, der wir lösen müssen.',
  'Das ist ein Problem, das wir lösen musst.'],
 array([0.6566717 , 0.4704884 , 0.24058492, 0.21603352, 0.20849238],
       dtype=float32))

### French to English

In [0]:
a = tfs.encode(["c'est un problème que nous devons résoudre."], model_file=encoding_model_file, add_bos=True, add_eos=True)[0]

In [0]:
ids, probs = predict(
    model=model,
    inputs=a,
    inpf=tf.constant([0.0, 1.0, 0.0, 0.0]),
    tarf=tf.constant([1.0, 0.0, 0.0, 0.0]),
    bos_id=1,
    eos_id=2,
    beam_size=5,
    vocab_size=40960,
    alpha=1.0,
)

In [0]:
mask = tf.cast(tf.not_equal(ids, 0), tf.int32)
seq_len = tf.reduce_sum(mask, axis=-1)
probs = tf.math.exp(probs)

In [0]:
ids_, probs_, seq_len_ = sess.run([ids, probs, seq_len])

In [25]:
[x.decode("utf-8") for x in sess.run(tfs.decode(ids_[0], seq_len_[0], model_file=encoding_model_file))], probs_[0]

(["That's a problem we need to resolve it.",
  "That's a problem we need to solve it.",
  'This is a problem we need to resolve it.',
  'That’s a problem we need to resolve it.',
  'That’s a problem we need to solve it.'],
 array([0.20478117, 0.19986515, 0.19323574, 0.18281902, 0.18169725],
       dtype=float32))

### German to English

In [0]:
a = tfs.encode(["Hier finden Sie alle Antworten, die Sie brauchen."], model_file=encoding_model_file, add_bos=True, add_eos=True)[0]

In [0]:
ids, probs = predict(
    model=model,
    inputs=a,
    inpf=tf.constant([0.0, 1.0, 0.0, 0.0]),
    tarf=tf.constant([1.0, 0.0, 0.0, 0.0]),
    bos_id=1,
    eos_id=2,
    beam_size=5,
    vocab_size=40960,
    alpha=1.0,
)

In [0]:
mask = tf.cast(tf.not_equal(ids, 0), tf.int32)
seq_len = tf.reduce_sum(mask, axis=-1)
probs = tf.math.exp(probs)

In [0]:
ids_, probs_, seq_len_ = sess.run([ids, probs, seq_len])

In [30]:
[x.decode("utf-8") for x in sess.run(tfs.decode(ids_[0], seq_len_[0], model_file=encoding_model_file))], probs_[0]

(['Here you will find all the answers you need.',
  'Here you will find all the answer you need.',
  'Here you find all the answers you need.',
  'You will find all the answers you need.',
  'You will find all the answers you need here.'],
 array([0.43479517, 0.2945328 , 0.23725006, 0.19760388, 0.18721606],
       dtype=float32))

### French to German

__NOTE:__ This is zero-shot, training was done for en-fr and en-de

In [0]:
a = tfs.encode(["c'est un problème que nous devons résoudre."], model_file=encoding_model_file, add_bos=True, add_eos=True)[0]

In [0]:
ids, probs = predict(
    model=model,
    inputs=a,
    inpf=tf.constant([0.0, 1.0, 0.0, 0.0]),
    tarf=tf.constant([0.0, 0.0, 1.0, 0.0]),
    bos_id=1,
    eos_id=2,
    beam_size=5,
    vocab_size=40960,
    alpha=1.0,
)

In [0]:
mask = tf.cast(tf.not_equal(ids, 0), tf.int32)
seq_len = tf.reduce_sum(mask, axis=-1)
probs = tf.math.exp(probs)

In [0]:
ids_, probs_, seq_len_ = sess.run([ids, probs, seq_len])

In [35]:
[x.decode("utf-8") for x in sess.run(tfs.decode(ids_[0], seq_len_[0], model_file=encoding_model_file))], probs_[0]

(['Das ist ein Problem, das wir beheben müssen.',
  'Es ist ein Problem, das wir beheben müssen.',
  'Das ist ein Problem, den wir beheben müssen.',
  'Das ist ein Problem, das wir lösen müssen.',
  'Das ist ein Problem, das wir zu lösen müssen.'],
 array([0.43660545, 0.32214886, 0.19762374, 0.17219017, 0.10422194],
       dtype=float32))

### German to French

__NOTE:__ This is zero-shot, training was done for en-fr and en-de

In [0]:
a = tfs.encode(["Hier finden Sie alle Antworten, die Sie brauchen."], model_file=encoding_model_file, add_bos=True, add_eos=True)[0]

In [0]:
ids, probs = predict(
    model=model,
    inputs=a,
    inpf=tf.constant([0.0, 0.0, 1.0, 0.0]),
    tarf=tf.constant([0.0, 1.0, 0.0, 0.0]),
    bos_id=1,
    eos_id=2,
    beam_size=5,
    vocab_size=40960,
    alpha=1.0,
)

In [0]:
mask = tf.cast(tf.not_equal(ids, 0), tf.int32)
seq_len = tf.reduce_sum(mask, axis=-1)
probs = tf.math.exp(probs)

In [0]:
ids_, probs_, seq_len_ = sess.run([ids, probs, seq_len])

In [40]:
[x.decode("utf-8") for x in sess.run(tfs.decode(ids_[0], seq_len_[0], model_file=encoding_model_file))], probs_[0]

(['Vous trouverez ici toutes les réponses que vous avez besoin.',
  'Vous trouverez ici toutes les réponses dont vous avez besoin.',
  'Vous trouverez tous les réponses que vous avez besoin.',
  'Vous trouverez tous les réponses dont vous avez besoin.',
  'Ici vous trouver toutes les réponses que vous avez besoin.'],
 array([0.33657932, 0.3270293 , 0.32325435, 0.29828098, 0.28019544],
       dtype=float32))

In [0]:
sess.close()